# 深層学習モデルを用いたクイズの自動生成

与えられた単語が答えとなる問題文を日本語T5モデルを用いて自動生成する。
- 入力: 答えとなる単語と、答えが含まれているパラグラフ（例えば、Wikipediaの一つのパラグラフ）
- 出力: 問題文

## ライブラリをインストールする。

In [1]:
!pip install -q transformers==4.26.1 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 6.0 MB/s eta 0:00:00


## 学習済み日本語T5モデルを利用して問題文を生成する

- T5（Text-to-Text Transfer Transformer）とは、機械翻訳や文書要約、質問回答、分類タスク（感情分析など）などの様々な自然言語処理タスクに転移学習させることができる深層学習モデルである。タスクの入出力をある種のテキスト形式で表現することにより、共通の事前学習モデルとロス関数、ハイパーパラメータを用いて転移学習させることができる。  
今回は、問題文を生成するというタスクに転移学習させた学習済みの日本語T5モデルを利用する。
- T5: https://ai.googleblog.com/2020/02/exploring-transfer-learning-with-t5.html
- T5の日本語解説記事: https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part7.html
- 問題文を生成する日本語T5モデル: https://huggingface.co/sonoisa/t5-base-japanese-question-generation

## 問題文生成モデルを読み込む。

In [2]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name_or_path = "sonoisa/t5-base-japanese-question-generation"

model = T5ForConditionalGeneration.from_pretrained(model_name_or_path)
model.eval()
if torch.cuda.is_available():
  model.cuda()

tokenizer = T5Tokenizer.from_pretrained(model_name_or_path, is_fast=True)

## 問題文を生成してみる。

* **モデルの入力**: "answer: *答えとなる単語* context: *答えが含まれているパラグラフ*"という形式の文字列をSentencePieceを用いてトークナイズしたトークン列
* **モデルの出力**: "*問題文*"のトークン列

In [3]:
from tqdm.auto import tqdm

INPUT_MAX_LEN = 512  # モデルに入力されるトークン列の最大長。最大長を超えたトークンは切り捨てられる。
OUTPUT_MAX_LEN = 64  # モデルから出力されるトークン列の最大長。最大長を超えないように文が生成されるはず。

answer_context_list = []

# 出典: https://ja.wikipedia.org/wiki/アマビエ
context = "アマビエ（歴史的仮名遣：アマビヱ）は、日本に伝わる半人半魚の妖怪。光輝く姿で海中から現れ、豊作や疫病などの予言をすると伝えられている。江戸時代後期の肥後国（現・熊本県）に現れたという。この話は挿図付きで瓦版に取り上げられ、遠く江戸にまで伝えられた。弘化3年4月中旬（1846年5月上旬）のこと、毎夜、海中に光る物体が出没していたため、役人が赴いたところ、それが姿を現した。姿形について言葉では書き留められていないが、挿図が添えられている。 その者は、役人に対して「私は海中に住むアマビエと申す者なり」と名乗り、「当年より6ヶ年の間は諸国で豊作が続くが疫病も流行する。私の姿を描いた絵を人々に早々に見せよ。」と予言めいたことを告げ、海の中へと帰って行った。年代が特定できる最古の例は、天保15年（1844年）の越後国（現・新潟県）に出現した「海彦（読みの推定：あまびこ）」を記述した瓦版（『坪川本』という。福井県立図書館所蔵）、その挿絵に描かれた海彦は、頭からいきなり3本の足が生えた（胴体のない）形状で、人間のような耳をし、目はまるく、口が突出している。その年中に日本人口の7割の死滅を予言し、その像の絵札による救済を忠告している。"
for answer in ["アマビエ", "豊作や疫病など", "肥後国（現・熊本県）", "当年より6ヶ年", "私の姿を描いた絵", "天保15年（1844年）", "3本", "7割"]:
  answer_context_list.append((answer, context))

generated_questions = []

for answer, context in tqdm(answer_context_list):
  # モデルに入力可能な形式に変換する。
  input = f"answer: {answer} context: {context}"

  # 入力文をトークナイズする。
  tokenized_inputs = tokenizer.batch_encode_plus(
      [input], max_length=INPUT_MAX_LEN, truncation=True, 
      padding="longest", return_tensors="pt")

  input_ids = tokenized_inputs['input_ids']
  input_mask = tokenized_inputs['attention_mask']
  if torch.cuda.is_available():
    input_ids = input_ids.cuda()
    input_mask = input_mask.cuda()

  # 問題文を生成する。
  tokenized_outputs = model.generate(input_ids=input_ids, attention_mask=input_mask, 
    max_length=OUTPUT_MAX_LEN, return_dict_in_generate=True, decoder_start_token_id=0,
    temperature=0.0,  # 生成にランダム性を入れる温度パラメータ
    num_beams=4,  # ビームサーチの探索幅
    # diversity_penalty=1.0,  # 生成結果の多様性を生み出すためのペナルティパラメータ
    # num_beam_groups=4,  # ビームサーチのグループ
    num_return_sequences=1,  # 生成する文の数
    )

  # 生成された問題文のトークン列を文字列に変換する。
  outputs = [tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) 
    for ids in tokenized_outputs.sequences]

  generated_questions.append(outputs)


  0%|          | 0/8 [00:00<?, ?it/s]

## 生成結果を確認してみる。

In [4]:
import textwrap

for (answer, context), questions in zip(answer_context_list, generated_questions):
  print(f"answer: {answer}")
  # print("\n".join(textwrap.wrap(f"context: {context}")))
  for question in questions:
    print(f"  -> {question}")
  print()

answer: アマビエ
  -> 半人半魚の妖怪の名前は何ですか?

answer: 豊作や疫病など
  -> アマビエは海中から何を予言すると言われていますか?

answer: 肥後国（現・熊本県）
  -> 江戸時代のどの国にアマビエが現れたのですか?

answer: 当年より6ヶ年
  -> 諸国で豊作が続くが疫病も流行するのはどのくらいの期間ですか?

answer: 私の姿を描いた絵
  -> 瓦版に描かれたアマビエの絵は何ですか?

answer: 天保15年（1844年）
  -> 海彦が初めて登場したのはいつですか?

answer: 3本
  -> 「海彦」には足がいくつありましたか?

answer: 7割
  -> 海彦は、その年の日本人口の何パーセントの死を予言したのだろうか

